In [8]:
pip install openai

In [22]:
import openai
import json

In [24]:
def read_api_key(filepath):
    with open(filepath, 'r') as file:
        api_key_info = json.load(file)
    return api_key_info

In [30]:
from google.colab import files
uploaded = files.upload()

Saving APIKey.txt to APIKey.txt


In [63]:

for filename in uploaded.keys():
    print(f" {filename} is uploaded")

with open('APIKey.txt', 'r') as file:
    content = file.read()


 APIKey.txt is uploaded


In [ ]:
print(content)

In [35]:
openai.api_key =content

In [38]:
chat_completion = openai.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Say this is a test",
        }
    ],
    model="gpt-4o",
)


In [39]:
chat_completion

ChatCompletion(id='chatcmpl-AnGtZjtCpa9Mvi8HkepmQDjoaLmDK', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='This is a test. How can I assist you further?', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1736306381, model='gpt-4o-2024-08-06', object='chat.completion', service_tier=None, system_fingerprint='fp_b7d65f1a5b', usage=CompletionUsage(completion_tokens=13, prompt_tokens=12, total_tokens=25, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [67]:
import asyncio

async def fetch_data():
    print("stat gather data...")
    await asyncio.sleep(10)  #

    print("data gather finished")
    return "data"

def test():
    print("test mid data")

async def main():
    print("start main...")
    data = await fetch_data()  #
    print(f"accept data: {data}")




In [68]:
await main()

start main...
stat gather data...
data gather finished
accept data: data


In [51]:
completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Can you generate an example json object describing a fruit?",
        }
    ],
    model="gpt-4o",
    response_format={"type": "json_object"},
)

In [52]:
print(completion)

ChatCompletion(id='chatcmpl-AnJXnb0UgU4wC9GBSFmZflkfJkPFw', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='\n{\n  "name": "Apple",\n  "color": "Red",\n  "varieties": [\n    "Gala",\n    "Fuji",\n    "Granny Smith",\n    "Honeycrisp"\n  ],\n  "nutritional_info": {\n    "calories": 52,\n    "carbohydrates": 14,\n    "fiber": 2.4,\n    "sugars": 10,\n    "protein": 0.3,\n    "fat": 0.2\n  },\n  "season": "Fall",\n  "origin": "Central Asia",\n  "description": "Apples are sweet, crunchy fruits that are popular worldwide for their delicious flavor and nutritional benefits.",\n  "uses": [\n    "Eating fresh",\n    "Making cider",\n    "Baking pies",\n    "Cooking sauces"\n  ]\n}', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1736316563, model='gpt-4o-2024-08-06', object='chat.completion', service_tier=None, system_fingerprint='fp_5f20662549', usage=CompletionUsage(completion_tokens=178, prompt

In [55]:
print(completion.choices[0])

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='\n{\n  "name": "Apple",\n  "color": "Red",\n  "varieties": [\n    "Gala",\n    "Fuji",\n    "Granny Smith",\n    "Honeycrisp"\n  ],\n  "nutritional_info": {\n    "calories": 52,\n    "carbohydrates": 14,\n    "fiber": 2.4,\n    "sugars": 10,\n    "protein": 0.3,\n    "fat": 0.2\n  },\n  "season": "Fall",\n  "origin": "Central Asia",\n  "description": "Apples are sweet, crunchy fruits that are popular worldwide for their delicious flavor and nutritional benefits.",\n  "uses": [\n    "Eating fresh",\n    "Making cider",\n    "Baking pies",\n    "Cooking sauces"\n  ]\n}', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))


In [60]:
response_content = completion.choices[0].message.content
fruit_data = json.loads(response_content)

formatted_json = json.dumps(fruit_data, indent=4, ensure_ascii=False)
print(formatted_json)

{
    "name": "Apple",
    "color": "Red",
    "varieties": [
        "Gala",
        "Fuji",
        "Granny Smith",
        "Honeycrisp"
    ],
    "nutritional_info": {
        "calories": 52,
        "carbohydrates": 14,
        "fiber": 2.4,
        "sugars": 10,
        "protein": 0.3,
        "fat": 0.2
    },
    "season": "Fall",
    "origin": "Central Asia",
    "description": "Apples are sweet, crunchy fruits that are popular worldwide for their delicious flavor and nutritional benefits.",
    "uses": [
        "Eating fresh",
        "Making cider",
        "Baking pies",
        "Cooking sauces"
    ]
}


In [76]:
completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "중국 도시 10가지 json형태로 한국어로 나열해줘?",
        }
    ],
    model="gpt-4o",
    response_format={"type": "json_object"},
)

In [77]:
response_content = completion.choices[0].message.content
fruit_data = json.loads(response_content)

formatted_json = json.dumps(fruit_data, indent=4, ensure_ascii=False)
print(formatted_json)

{
    "도시들": [
        {
            "이름": "베이징"
        },
        {
            "이름": "상하이"
        },
        {
            "이름": "광저우"
        },
        {
            "이름": "선전"
        },
        {
            "이름": "청두"
        },
        {
            "이름": "충칭"
        },
        {
            "이름": "시안"
        },
        {
            "이름": "우한"
        },
        {
            "이름": "항저우"
        },
        {
            "이름": "텐진"
        }
    ]
}
